# Wave amplitude blowing up for large `t_total`. #6

This ipython notebook reproduces the bug discussed in `github` issue [Quazartech/DG_Maxwell Issue #6](https://github.com/QuazarTech/DG_Maxwell/issues/6)
in which the wave function was increasing with time and eventually blowing up exponentially after some time
$dt$.

In this notebook I have re-written the functions for calculating the $A$ matrix, $b$ matrix
and time evolution function using the already tested basic functions.

### Note
Use the following command to get the documentation of a function
```
    <function>?
```
For example, if you want to look at the documentation of the function `gvar.populateGlobalVariables`, in an empty
cell type `gvar.populateGlobalVariables?`.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from tqdm import trange
import arrayfire as af
# Available backends with arrayfire
# 1. cuda
# 2. opencl
# 3. cpu
af.set_backend('cuda')


# Contains the global variables for the functions.
from app import global_variables as gvar
# Contains the functions for 1D wave equation solver.
from app import wave_equation

plt.rcParams['figure.figsize'] = 9.6, 6.
plt.rcParams['figure.dpi'] = 100
plt.rcParams['image.cmap'] = 'jet'
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'serif'
plt.rcParams['text.usetex'] = True
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['axes.titlesize'] = 'medium'
plt.rcParams['axes.labelsize'] = 'medium'
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['xtick.minor.size'] = 4
plt.rcParams['xtick.major.pad'] = 8
plt.rcParams['xtick.minor.pad'] = 8
plt.rcParams['xtick.color'] = 'k'
plt.rcParams['xtick.labelsize'] = 'medium'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['ytick.minor.size'] = 4
plt.rcParams['ytick.major.pad'] = 8
plt.rcParams['ytick.minor.pad'] = 8
plt.rcParams['ytick.color'] = 'k'
plt.rcParams['ytick.labelsize'] = 'medium'
plt.rcParams['ytick.direction'] = 'in'


In [2]:
gvar.populateGlobalVariables(8, 10)

In [ ]:
A_inverse = af.lapack.inverse(wave_equation.A_matrix())
element_LGL = gvar.element_LGL
delta_t     = gvar.delta_t
for t_n in trange(0, gvar.time.shape[0] - 1):
    gvar.u[:, :, t_n + 1] = gvar.u[:, :, t_n] + af.blas.matmul(A_inverse,
                                                               wave_equation.b_vector(t_n))


In [ ]:
approximate_1_s       = (int(1 / gvar.delta_t) * gvar.delta_t)
analytical_u_after_1s = np.e ** (-(gvar.element_LGL - gvar.c
                                * (1 - approximate_1_s)) ** 2 / 0.4 ** 2)

af.display(analytical_u_after_1s, 10)
af.display(gvar.u[:, :, int(1 / gvar.delta_t)], 10)
af.display(gvar.u[:, :, 0], 10)

subprocess.run(['mkdir', 'results/1D_Wave_images'])

for t_n in trange(0, gvar.time.shape[0] - 1):
    if t_n % 100 == 0:
        fig = plt.figure()
        x   = gvar.element_LGL
        y   = gvar.u[:, :, t_n]

        plt.plot(x, y)
        plt.xlabel('x')
        plt.ylabel('Amplitude')
        plt.title('Time = %f' % (t_n * delta_t))
        fig.savefig('results/1D_Wave_images/%04d' %(t_n / 100) + '.png')
        plt.close('all')


To calculate $L_{p}(\xi) L_{i}(\xi)$ for N Lagrange-Gaussian-Lobatto points 
in a $\xi$ space [-1, 1] and indices i and p ranging from 0 to N - 1, We 
need to start by creating a 1 X N x N matrix of $L_{p}(\xi)$.

\begin{align}
L_{p}(\xi) = \begin{pmatrix}
L_{0}(\xi_{0}) & L_{1}(\xi_{0}) & \cdots & L_{N - 1}(\xi_{0}) \\
L_{0}(\xi_{1}) & L_{1}(\xi_{1}) & \cdots & L_{N - 1}(\xi_{1}) \\
\vdots  & \vdots  & \ddots & \vdots  \\
L_{0}(\xi_{N - 1}) & L_{1}(\xi_{N - 1}) & \cdots & L_{N - 1}(\xi_{N - 1}) 
\end{pmatrix}_{1 X N X N }
\end{align}

Now, we need to get a $L_{i}(\xi)$ matrix which is the transpose of $L_{p}
(\xi)$ and reordered as \newline(1, 0, 2), where (0, 1, 2) is the normal 
ordering (x, y, z)

\begin{align}
L_{i}(\xi) = \begin{pmatrix}
L_{0}(\xi_{0}) & L_{0}(\xi_{1}) & \cdots & L_{0}(\xi_{N - 1}) \\
L_{1}(\xi_{0}) & L_{1}(\xi_{1}) & \cdots & L_{1}(\xi_{N - 1}) \\
\vdots  & \vdots  & \ddots & \vdots  \\
L_{N - 1}(\xi_{0}) & L_{N - 1}(\xi_{1}) & \cdots & L_{N - 1}(\xi_{N - 1}) 
\end{pmatrix}_{N X 1 X N}
\end{align}

Now, the $L_{i}(\xi)$ array is broadcasted across the dimension 0 N times.
The result is an\newline N X N X N matrix with elements repeating N times 
in dimension 0.
\begin{tikzpicture}[every node\.style={anchor=north east,fill=white, 
minimum width=1.4cm, minimum height=7mm}]
\matrix (mA) [draw,matrix of math nodes]
{
L_{0}(\xi_{N - 1}) & L_{1}(\xi_{N - 1}) & ... & L_{N - 1}(\xi_{N - 1}) \\
L_{0}(\xi_{N - 1}) & L_{1}(\xi_{N - 1}) & ... & L_{N - 1}(\xi_{N - 1}) \\
. & . & ... & . \\
L_{0}(\xi_{N - 1}) & L_{1}(\xi_{N - 1}) & ... & L_{N - 1}(\xi_{N - 1}) \\
};
\matrix (mB) [draw,matrix of math nodes] at ($(mA.south west)+(2,0.7)$)
{
L_{0}(\xi_{i}) & L_{1}(\xi_{i}) & ... & L_{N - 1}(\xi_{i}) \\
L_{0}(\xi_{i}) & L_{1}(\xi_{i}) & ... & L_{N - 1}(\xi_{i}) \\
. & . & ... & . \\
L_{0}(\xi_{i}) & L_{1}(\xi_{i}) & ... & L_{N - 1}(\xi_{i}) \\
};
\matrix (mC) [draw,matrix of math nodes] at ($(mB.south west)+(2,0.7)$)
{
L_{0}(\xi_{0}) & L_{1}(\xi_{0}) & ... & L_{N - 1}(\xi_{0}) \\
L_{0}(\xi_{0}) & L_{1}(\xi_{0}) & ... & L_{N - 1}(\xi_{0}) \\
. & . & ... & . \\
L_{0}(\xi_{0}) & L_{1}(\xi_{0}) & ... & L_{N - 1}(\xi_{0}) \\
};
\draw[dashed](mA.north east)--(mC.north east);
\draw[dashed](mA.north west)--(mC.north west);
\draw[dashed](mA.south east)--(mC.south east);
\end{tikzpicture}
Now doing the same for $L_{i}(\xi)$, i.e., broadcasting it in dimension 1 
N times. Another \newline N X N X N array would be obtained which can be 
multipled with the previous N X N X N array which would give the required
$L_{p}(\xi) L_{i}(\xi)$ matrix
\begin{tikzpicture}[every node\.style={anchor=north east,fill=white, 
minimum width=1.4cm,minimum 		height=7mm}]
\matrix (mA) [draw,matrix of math nodes]
{
L_{0}(\xi_{N -1}) & L_{0}(\xi_{N -1}) & ... & L_{0}(\xi_{N -1}) \\
L_{1}(\xi_{N -1}) & L_{1}(\xi_{N -1}) & ... & L_{1}(\xi_{N -1}) \\
. & . & ... & . \\
L_{N - 1}(\xi_{N -1}) & L_{N - 1}(\xi_{N -1}) & ... & L_{N - 1}(\xi_{N -1})
\\
};
\matrix (mB) [draw,matrix of math nodes] at ($(mA.south west)+(2,0.7)$)
{
L_{0}(\xi_{i}) & L_{0}(\xi_{i}) & ... & L_{0}(\xi_{i}) \\
L_{1}(\xi_{i}) & L_{1}(\xi_{i}) & ... & L_{1}(\xi_{i}) \\
. & . & ... & . \\
L_{N - 1}(\xi_{i}) & L_{N - 1}(\xi_{i}) & ... & L_{N - 1}(\xi_{i}) \\
};
\matrix (mC) [draw,matrix of math nodes] at ($(mB.south west)+(2,0.7)$)
{
L_{0}(\xi_{0}) & L_{0}(\xi_{0}) & ... & L_{0}(\xi_{0}) \\
L_{1}(\xi_{0}) & L_{1}(\xi_{0}) & ... & L_{1}(\xi_{0}) \\
. & . & ... & . \\
L_{N - 1}(\xi_{0}) & L_{N - 1}(\xi_{0}) & ... & L_{N - 1}(\xi_{0}) \\
};
\draw[dashed](mA.north east)--(mC.north east);
\draw[dashed](mA.north west)--(mC.north west);
\draw[dashed](mA.south east)--(mC.south east);
\end{tikzpicture}
\begin{tikzpicture}[every node\.style={anchor=north east,fill=white, 
minimum width=1.4cm,minimum height=10mm}]
\matrix (mA) [draw,matrix of math nodes]
{
L_{0}(\xi_{N - 1})L_{0}(\xi_{N - 1}) & L_{0}(\xi_{N - 1})L_{1}
(\xi_{N - 1}) & ... & L_{0}(\xi_{N - 1})L_{N - 1}(\xi_{N - 1}) \\
L_{1}(\xi_{N - 1})L_{0}(\xi_{N - 1}) & L_{1}(\xi_{N - 1})L_{1}
(\xi_{N - 1}) & ... & L_{1}(\xi_{N - 1})L_{N - 1}(\xi_{N - 1}) \\
. & . & ... & . \\
L_{N - 1}(\xi_{N - 1})L_{0}(\xi_{N - 1}) & L_{N - 1}
(\xi_{N - 1})L_{1}(\xi_{N - 1}) & ... & L_{N - 1}(\xi_{N - 1})L_{N - 1}
(\xi_{N - 1}) \\
};
\matrix (mB) [draw,matrix of math nodes] at ($(mA.south west)+(1.5,0.7)$)
{
L_{0}(\xi_{i})L_{0}(\xi_{i}) & L_{0}(\xi_{i})L_{1}(\xi_{i}) & ... & L_{0}
(\xi_{i})L_{N - 1}(\xi_{i}) \\
L_{1}(\xi_{i})L_{0}(\xi_{i}) & L_{1}(\xi_{i})L_{1}(\xi_{i}) & ... & L_{1}
(\xi_{i})L_{N - 1}(\xi_{i}) \\
. & . & ... & . \\
L_{N - 1}(\xi_{i})L_{i}(\xi_{i}) & L_{N - 1}(\xi_{i})L_{1}(\xi_{i}) & ... 
& L_{N - 1}(\xi_{i})L_{N - 1}(\xi_{i}) \\
};
\matrix (mC) [draw,matrix of math nodes] at ($(mB.south west)+(1.5,0.7)$)
{
L_{0}(\xi_{0})L_{0}(\xi_{0}) & L_{0}(\xi_{0})L_{1}(\xi_{0}) & ... & L_{0}
(\xi_{0})L_{N - 1}(\xi_{0}) \\
L_{1}(\xi_{0})L_{0}(\xi_{0}) & L_{1}(\xi_{0})L_{1}(\xi_{0}) & ... & L_{1}
(\xi_{0})L_{N - 1}(\xi_{0}) \\
. & . & ... & . \\
L_{N - 1}(\xi_{0})L_{0}(\xi_{0}) & L_{N - 1}(\xi_{0})L_{1}(\xi_{0}) & 
... & L_{N - 1}(\xi_{0})L_{N - 1}(\xi_{0}) \\
};
\draw[dashed](mA.north east)--(mC.north east);
\draw[dashed](mA.north west)--(mC.north west);
\draw[dashed](mA.south east)--(mC.south east);
\end{tikzpicture}